In [1]:
### Structure
# Extract sample data from BigQuery
# Exploring data
# Quality control and other preprocessing
# Create ML datasets
# Verify that datasets exist
# Benchmark
# Benchmark on same dataset

In [35]:
# Extract sample data from BigQuery
# Import libraries
from google.cloud import bigquery
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import shutil

In [36]:
# SQL query
sql = """
    SELECT
        pickup_datetime, pickup_longitude, pickup_latitude, dropoff_longitude, 
        dropoff_latitude, passenger_count, trip_distance, tolls_amount, 
        fare_amount, total_amount 
    FROM `nyc-tlc.yellow.trips`
    LIMIT 10
"""

In [37]:
# Run a query
CREDS = 'arboreal-parser-228610-9d3cc71c229e.json'
client = bigquery.Client.from_service_account_json(json_credentials_path=CREDS)
trips = client.query(sql).to_dataframe()
trips

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,trip_distance,tolls_amount,fare_amount,total_amount
0,2015-04-17 08:07:24+00:00,-73.901932,40.764160,-73.902008,40.764130,5,0.0,0.0,0.0,0.0
1,2010-02-06 20:23:08+00:00,-73.795666,40.644648,-73.980812,40.764683,2,178.7,0.0,0.0,0.0
2,2010-02-02 23:17:55+00:00,-73.924135,40.752351,-73.924135,40.752351,1,0.0,0.0,0.0,0.0
3,2013-08-08 15:21:50+00:00,-74.000860,40.762344,-74.000997,40.762137,1,0.0,0.0,0.0,0.0
4,2010-02-18 12:29:13+00:00,-73.976301,40.732448,-73.976340,40.732448,1,0.0,0.0,0.0,0.0
5,2010-03-16 17:13:04+00:00,-73.971133,40.760423,-73.786367,40.638546,1,16.1,0.0,0.0,0.0
6,2010-03-04 04:25:37+00:00,-73.776251,40.645725,-73.776251,40.645725,1,0.0,0.0,0.0,0.0
7,2013-08-31 23:59:24+00:00,-73.995712,40.723319,-74.001071,40.727537,2,4.4,0.0,0.0,0.0
8,2015-04-12 05:38:51+00:00,-73.937500,40.758331,-73.937523,40.758335,1,0.0,0.0,0.0,0.0
9,2015-05-24 23:47:47+00:00,-73.990479,40.757786,-74.023338,40.759418,2,3.0,0.0,0.0,0.3
